<div style="display: flex; align-items: center; justify-content: center; flex-wrap: wrap;">
    <div style="flex: 1; min-width: 250px; display: flex; justify-content: center;">
        <img src="https://adnova.novaims.unl.pt/media/22ui3ptm/logo.svg" style="max-width: 80%; height: auto; margin-top: 50px; margin-bottom: 50px;margin-left: 3rem;">
    </div>
    <div style="flex: 2; text-align: center; margin-top: 20px;margin-left: 8rem;">
        <div style="font-size: 28px; font-weight: bold; line-height: 1.2;">
            <span style='color:#6f800f'> Data Mining Project | </span>
            <span style='color:#393B79'>ABCDEats Inc.</span>
        </div>
        <div style="font-size: 17px; font-weight: bold; margin-top: 10px;">
            Fall Semester | 2024 - 2025
        </div>
        <div style="font-size: 17px; font-weight: bold;">
            Master in Data Science and Advanced Analytics
        </div>
        <div style="margin-top: 20px;">
            <div>André Silvestre, 20240502</div>
            <div>Filipa Pereira, 20240509</div>
            <div>Umeima Mahomed, 20240543</div>
        </div>
        <div style="margin-top: 20px; font-weight: bold;">
            Group 37
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right,#6f800f, #393B79); 
            padding: .7px; color: white; border-radius: 300px; text-align: center;">
</div>

## 📚 Libraries Import

In [ ]:
# For data
import pandas as pd
import numpy as np
import os

# For plotting and EDA
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import matplotlib.lines as mlines
from matplotlib.colors import LinearSegmentedColormap

# For preprocessing
from sklearn.preprocessing import StandardScaler

# Set the style of the visualization
pd.set_option('display.max_columns', None)                  # display all columns
pd.set_option('display.float_format', lambda x: '%.2f' % x) # display floats with 2 decimal places

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Setting seaborn style
plt.style.use('ggplot')
sns.set_theme(style='white')

# <a class='anchor' id='2'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #6f800f,#393B79); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 140px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 32px; font-family: 'Avenir Next LT Pro', sans-serif;">
        <b>Part 2 | Data Preprocess </b></h1></center>
</div>

In [3]:
# Importing the dataset
ABCDEats = pd.read_parquet('data/DM2425_ABCDEats_1stPart.parquet')

In [4]:
# Display the first 5 rows just to confirm the import was successful
ABCDEats.head() 

Variables,customer_id,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,CUI_American,CUI_Asian,CUI_Beverages,CUI_Cafe,CUI_Chicken Dishes,CUI_Chinese,CUI_Desserts,CUI_Healthy,CUI_Indian,CUI_Italian,CUI_Japanese,CUI_Noodle Dishes,CUI_OTHER,CUI_Street Food / Snacks,CUI_Thai,DOW_0,DOW_1,DOW_2,DOW_3,DOW_4,DOW_5,DOW_6,HR_0,HR_1,HR_2,HR_3,HR_4,HR_5,HR_6,HR_7,HR_8,HR_9,HR_10,HR_11,HR_12,HR_13,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23,order_count,customer_region_buckets,customer_age_group,days_between_orders,days_between_orders_per_order,last_promo_bin,CUI_Total_Amount_Spent,CUI_Most_Spent_Cuisine,CUI_Total_Food_Types,CUI_Avg_Amount_Spent
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1b8f824d5e,2360,18.00,2,5,1,0.00,1,DELIVERY,DIGI,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,28.88,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0,0,0,0,1,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,2,15-28,1.00,0.50,1,28.88,Indian,1,14.44
1,5d272b9dcb,8670,17.00,2,2,2,0.00,1,DISCOUNT,DIGI,12.82,6.39,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0,0,0,0,1,0.00,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2,8,15-28,1.00,0.50,1,19.21,American,2,9.61
2,f6d1b2ba63,4660,38.00,1,2,2,0.00,1,DISCOUNT,CASH,9.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,0,0,0,0,0,1,0.00,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,4,29-41,1.00,0.50,1,9.20,American,1,4.60
3,180c632ed8,4660,NaN,2,3,1,0.00,2,DELIVERY,DIGI,0.00,13.70,0.00,0.00,0.00,0.00,0.00,0.00,17.86,0.00,0.00,0.00,0.00,0.00,0.00,0,1,0,0,0,0,1,0.00,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,2,4,NaN,2.00,1.00,1,31.56,Indian,2,15.78
4,4eb37a6705,4660,20.00,2,5,0,0.00,2,NO PROMO,DIGI,14.57,40.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,1,0,0,0,0,1,0.00,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4,15-28,2.00,1.00,0,55.44,Asian,2,27.72


In [5]:
# Number of rows and columns
print('Number of\033[1m rows \033[0m:', ABCDEats.shape[0])
print('Number of\033[1m columns \033[0m:', ABCDEats.shape[1])

Number of rows : 31737
Number of columns : 66


In [6]:
# Check the data types
ABCDEats.dtypes

Variables
customer_id                 object
customer_region           category
customer_age               float64
vendor_count                 int64
product_count                int64
                            ...   
last_promo_bin               int64
CUI_Total_Amount_Spent     float64
CUI_Most_Spent_Cuisine      object
CUI_Total_Food_Types         int64
CUI_Avg_Amount_Spent       float64
Length: 66, dtype: object

---

## **🛠️ Data Preprocessing/Feature Engineering**

In [8]:
# Create a continuous and discrete colormap
colors = ["#3E460F", "#4E5813", "#626E18", "#7A891E", "#98AB26", "#BED62F"]
NOVAIMS_palette_colors = sns.color_palette(colors[::-1], as_cmap=True)

colors = ["#3E460F", "#4E5813", "#626E18", "#7A891E", "#98AB26", "#BED62F", "#FFFFFF"]
NOVAIMS_palette_colors_continuous = LinearSegmentedColormap.from_list("NOVAIMS_palette", colors[::-1])

In [ ]:
# Define metric and non-metric features
metric_cols = ABCDEats.select_dtypes(include=['int64', 'float64']).columns
non_metric_cols = ABCDEats[:1].select_dtypes(include=['object']).columns             ########### last_promo_bin

# Exclude the column 'customer_id' from the Non-Metric columns
non_metric_cols = non_metric_cols.drop('customer_id')

print(f'Metric columns: {len(metric_cols)}, {metric_cols} \n')
print(f'Non-Metric columns: {len(non_metric_cols)}, {non_metric_cols}')

Metric columns: 59, Index(['customer_age', 'vendor_count', 'product_count', 'is_chain',
       'first_order', 'last_order', 'CUI_American', 'CUI_Asian',
       'CUI_Beverages', 'CUI_Cafe', 'CUI_Chicken Dishes', 'CUI_Chinese',
       'CUI_Desserts', 'CUI_Healthy', 'CUI_Indian', 'CUI_Italian',
       'CUI_Japanese', 'CUI_Noodle Dishes', 'CUI_OTHER',
       'CUI_Street Food / Snacks', 'CUI_Thai', 'DOW_0', 'DOW_1', 'DOW_2',
       'DOW_3', 'DOW_4', 'DOW_5', 'DOW_6', 'HR_0', 'HR_1', 'HR_2', 'HR_3',
       'HR_4', 'HR_5', 'HR_6', 'HR_7', 'HR_8', 'HR_9', 'HR_10', 'HR_11',
       'HR_12', 'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19',
       'HR_20', 'HR_21', 'HR_22', 'HR_23', 'order_count',
       'days_between_orders', 'days_between_orders_per_order',
       'last_promo_bin', 'CUI_Total_Amount_Spent', 'CUI_Total_Food_Types',
       'CUI_Avg_Amount_Spent'],
      dtype='object', name='Variables') 

Non-Metric columns: 4, Index(['last_promo', 'payment_method', 'customer_region_

In [10]:
# Unique values of the columns 'CUI_American', 'CUI_Asian', 'CUI_Chinese', 'CUI_Italian', etc.
cuisines_cols = [col for col in ABCDEats.columns if 'CUI_' in col]
cuisines_cols

['CUI_American',
 'CUI_Asian',
 'CUI_Beverages',
 'CUI_Cafe',
 'CUI_Chicken Dishes',
 'CUI_Chinese',
 'CUI_Desserts',
 'CUI_Healthy',
 'CUI_Indian',
 'CUI_Italian',
 'CUI_Japanese',
 'CUI_Noodle Dishes',
 'CUI_OTHER',
 'CUI_Street Food / Snacks',
 'CUI_Thai',
 'CUI_Total_Amount_Spent',
 'CUI_Most_Spent_Cuisine',
 'CUI_Total_Food_Types',
 'CUI_Avg_Amount_Spent']

In [12]:
# Weeekdays columns
weekdays_cols = ABCDEats.loc[:, 'DOW_0':'DOW_6'].columns

# Hours columns
hours_cols = ABCDEats.loc[:, 'HR_0':'HR_23'].columns

In [13]:
# List of weekdays (0 = Sunday, 6 = Saturday)
weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
weekdays_dict = dict(enumerate(weekdays))
weekdays_dict

{0: 'Sunday',
 1: 'Monday',
 2: 'Tuesday',
 3: 'Wednesday',
 4: 'Thursday',
 5: 'Friday',
 6: 'Saturday'}

---

In [19]:
# Define function for outlier detection, based on IQR method
def detect_outliers(df,features, dfoutput=False, info=True):
    """
    This function detects outliers based on the IQR method.
    
    :param df: DataFrame
    :param features: List of features to detect outliers
    :param dfoutput: Boolean to return only the DataFrame with the outliers (without printing the information)
    :param info: Boolean to print the information about the outliers
    :return: DataFrame with the outliers
    """
    
    # Check if features is a list
    if not isinstance(features, list):
        features = [features]
    
    if len(features) == 0:
        return []
    
    # Detect outliers in 1 features
    for feature in features:
        Q1 = df[feature].quantile(0.25)       # 1st quartile
        Q3 = df[feature].quantile(0.75)       # 3rd quartile
        IQR = Q3 - Q1                         # InterQuartile Range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
        
        # Print the information about the outliers
        if info:
            print('\033[1m', feature, '\033[0m',
                    '\n  1st Quartile:', Q1, '  | 3rd Quartile:', Q3, '  | IQR:', IQR,
                    '\n  Lower Bound:', lower_bound, '  | Upper Bound:', upper_bound,
                    '\n\n  Number of outliers:', len(outliers), '(', round(len(outliers)/len(df)*100, 2), '%)',
                    '\n    Min:', outliers[feature].min(), '  | Max:', outliers[feature].max(), '\n')
            
        # Return DataFrame with the outliers 
        if dfoutput:
            return outliers

In [20]:
# Apply the function 'detect_outliers' to all the metric columns
for col in metric_cols:
    detect_outliers(ABCDEats, col, info=True)

 customer_age  
  1st Quartile: 23.0   | 3rd Quartile: 31.0   | IQR: 8.0 
  Lower Bound: 11.0   | Upper Bound: 43.0 

  Number of outliers: 1078 ( 3.4 %) 
    Min: 44.0   | Max: 80.0 

 vendor_count  
  1st Quartile: 1.0   | 3rd Quartile: 4.0   | IQR: 3.0 
  Lower Bound: -3.5   | Upper Bound: 8.5 

  Number of outliers: 1523 ( 4.8 %) 
    Min: 9   | Max: 41 

 product_count  
  1st Quartile: 2.0   | 3rd Quartile: 7.0   | IQR: 5.0 
  Lower Bound: -5.5   | Upper Bound: 14.5 

  Number of outliers: 2370 ( 7.47 %) 
    Min: 15   | Max: 269 

 is_chain  
  1st Quartile: 1.0   | 3rd Quartile: 3.0   | IQR: 2.0 
  Lower Bound: -2.0   | Upper Bound: 6.0 

  Number of outliers: 3154 ( 9.94 %) 
    Min: 7   | Max: 83 

 first_order  
  1st Quartile: 7.0   | 3rd Quartile: 44.5   | IQR: 37.5 
  Lower Bound: -49.25   | Upper Bound: 100.75 

  Number of outliers: 0 ( 0.0 %) 
    Min: nan   | Max: nan 

 last_order  
  1st Quartile: 49.0   | 3rd Quartile: 83.0   | IQR: 34.0 
  Lower Bound: -2.0   | Up

---

### **PCA (Principal Component Analysis)**

---

# **💾 Save Data** 

<br>

- To finish this notebook and proceed Cluster Analysis, we will save the data to a new file.

In [ ]:
# Save the preprocessed dataset
#ABCDEats.to_parquet('data/DM2425_ABCDEats_preprocessed.parquet')